In [ ]:
!pip install https://github.com/matplotlib/mpl_finance/archive/master.zip

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

!pip install tushare

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import talib
import tushare as ts
from mpl_finance import candlestick_ohlc
from matplotlib.pylab import date2num
import math
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_predict
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost

In [ ]:
df = pd.read_csv('/content/EURUSD60.csv', names=['DATE','TIME','OPEN','HIGH','LOW','CLOSE','VOLUME'])
df['DATE'] = df['DATE'].str.replace('.','-')
df['DATE'] = df['DATE'] +'  ' + df['TIME']
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.rename(columns={'DATE':'date', 'OPEN':'open', 'HIGH':'high', 'LOW':'low', 'CLOSE':'close', 'VOLUME':'volume'})
df.index = df['date']
df = df.drop(['TIME'], axis=1)
df.head()

,date,open,high,low,close,volume
date,,,,,,
2009-10-08 06:00:00,2009-10-08 06:00:00,1.47670,1.47712,1.47529,1.47591,1408
2009-10-08 07:00:00,2009-10-08 07:00:00,1.47594,1.47634,1.47468,1.47550,1481
2009-10-08 08:00:00,2009-10-08 08:00:00,1.47553,1.47619,1.47507,1.47559,1541
2009-10-08 09:00:00,2009-10-08 09:00:00,1.47558,1.47675,1.47407,1.47628,2739
2009-10-08 10:00:00,2009-10-08 10:00:00,1.47629,1.47854,1.47618,1.47755,3300


In [ ]:
mpl.style.use("ggplot")
# Calculate MACD Indicator df
df["macd"], df["sigal"], df["hist"] = talib.MACD(df.close)

# Compute RSI
df["rsi"] = talib.RSI(df.close)

# Calculate moving average
df["ma10"] = talib.MA(df.close, timeperiod=10)
df["ma20"] = talib.MA(df.close, timeperiod=20)
df["ma15"] = talib.MA(df.close, timeperiod=15)

df['ma10/ma20'] = df['ma10'] / df['ma20']
df['ma10/ma15'] = df['ma10'] / df['ma15']
df['ma15/ma20'] = df['ma15'] / df['ma20']

df.head()

,date,open,high,low,close,volume,macd,sigal,hist,rsi,ma10,ma20,ma15,ma10/ma20,ma10/ma15,ma15/ma20
date,,,,,,,,,,,,,,,,
2009-10-08 06:00:00,2009-10-08 06:00:00,1.47670,1.47712,1.47529,1.47591,1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10-08 07:00:00,2009-10-08 07:00:00,1.47594,1.47634,1.47468,1.47550,1481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10-08 08:00:00,2009-10-08 08:00:00,1.47553,1.47619,1.47507,1.47559,1541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10-08 09:00:00,2009-10-08 09:00:00,1.47558,1.47675,1.47407,1.47628,2739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10-08 10:00:00,2009-10-08 10:00:00,1.47629,1.47854,1.47618,1.47755,3300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['ma10/ma20_diff'] = df['ma10/ma20'] - df['ma10/ma20'].shift(1)
df = df.sort_index()
df['ma10/ma15_diff'] = df['ma10/ma15'] - df['ma10/ma15'].shift(1) 
df = df.sort_index()
df['ma15/ma20_diff'] = df['ma15/ma20'] - df['ma15/ma20'].shift(1)
df = df.sort_index()
df['close_diff'] = df['close'] - df['close'].shift(1)
df = df.sort_index()
df['open_diff'] = df['open'] - df['open'].shift(1)
df = df.sort_index()
df['high_diff'] = df['high'] - df['high'].shift(1)
df = df.sort_index()
df['low_diff'] = df['low'] - df['low'].shift(1)
df = df.sort_index()
df['volume_diff'] = df['volume'] - df['volume'].shift(1)
df = df.sort_index()

df['ma10/ma20_difff'] = df['ma10/ma20'] - df['ma10/ma20'].shift(2)
df = df.sort_index()
df['ma10/ma15_difff'] = df['ma10/ma15'] - df['ma10/ma15'].shift(2) 
df = df.sort_index()
df['ma15/ma20_difff'] = df['ma15/ma20'] - df['ma15/ma20'].shift(2)
df = df.sort_index()
df['close_difff'] = df['close'] - df['close'].shift(2)
df = df.sort_index()
df['open_difff'] = df['open'] - df['open'].shift(2)
df = df.sort_index()
df['high_difff'] = df['high'] - df['high'].shift(2)
df = df.sort_index()
df['low_difff'] = df['low'] - df['low'].shift(2)
df = df.sort_index()
df['volume_difff'] = df['volume'] - df['volume'].shift(2)
df = df.sort_index()

df[['close','close_diff','close_difff']].head(4)

,close,close_diff,close_difff
date,,,
2009-10-08 06:00:00,1.47591,NaN,NaN
2009-10-08 07:00:00,1.47550,-0.00041,NaN
2009-10-08 08:00:00,1.47559,0.00009,-0.00032
2009-10-08 09:00:00,1.47628,0.00069,0.00078


In [ ]:
lag = 3
df['mx'] = df['high'].rolling(lag).max().shift(-lag)
df['mn'] = df['low'].rolling(lag).min().shift(-lag)
df['diff_high'] = df['mx'] - df['close']
df['diff_low'] = df['close'] - df['mn']

gap = 0.002
df['target'] = 'so so'
df['target'][df['diff_high']>=gap] = 'buy'
df['target'][df['diff_low']>=gap] = 'sell'
df['target'][(df['diff_high']>=gap) & (df['diff_low']>=gap)] = 'both'
# df['target'][(df['diff_high']>=1.5*gap)] = 'buy'
# df['target'][(df['diff_low']>=1.5*gap)] = 'sell'

print(df['target'].value_counts())
print(df.head(10))

so so    32782
buy      14579
sell     14510
both      3421
Name: target, dtype: int64
                                   date     open  ...  diff_low  target
date                                              ...                  
2009-10-08 06:00:00 2009-10-08 06:00:00  1.47670  ...   0.00184   so so
2009-10-08 07:00:00 2009-10-08 07:00:00  1.47594  ...   0.00143     buy
2009-10-08 08:00:00 2009-10-08 08:00:00  1.47553  ...   0.00152     buy
2009-10-08 09:00:00 2009-10-08 09:00:00  1.47558  ...   0.00031     buy
2009-10-08 10:00:00 2009-10-08 10:00:00  1.47629  ...   0.00158   so so
2009-10-08 11:00:00 2009-10-08 11:00:00  1.47757  ...   0.00158   so so
2009-10-08 12:00:00 2009-10-08 12:00:00  1.47677  ...   0.00223    both
2009-10-08 13:00:00 2009-10-08 13:00:00  1.47660  ...   0.00430    both
2009-10-08 14:00:00 2009-10-08 14:00:00  1.47669  ...   0.00343    both
2009-10-08 15:00:00 2009-10-08 15:00:00  1.47527  ...   0.00744    sell

[10 rows x 37 columns]


In [ ]:
df = df[df.target!='both']
print(df.shape)
df = df.dropna(axis=0)
print(df.shape)
# df['target'][df['target']==-1] = np.nan
# df['target'] = df['target'].bfill()
# df = df[~df['target'].isnull()]
print(df['target'].value_counts())

(61871, 37)
(61840, 37)
so so    32771
buy      14568
sell     14501
Name: target, dtype: int64


In [ ]:
X = df[['high','close','low','open', 'volume','macd','sigal','hist','ma10','ma15','rsi', 'ma20',	'ma10/ma20',	'ma10/ma15',	'ma15/ma20',	'ma10/ma15_diff',	'ma10/ma20_diff',	'ma15/ma20_diff','close_diff',	'open_diff',	'high_diff',	'low_diff',	'volume_diff',	'ma10/ma20_difff',	'ma10/ma15_difff',	'ma15/ma20_difff',	'close_difff',	'open_difff',	'high_difff',	'low_difff',	'volume_difff']]
Y = df['target']

# X = X.iloc[40:,:]
# Y = Y.iloc[40:]
X.head()

,high,close,low,open,volume,macd,sigal,hist,ma10,ma15,rsi,ma20,ma10/ma20,ma10/ma15,ma15/ma20,ma10/ma15_diff,ma10/ma20_diff,ma15/ma20_diff,close_diff,open_diff,high_diff,low_diff,volume_diff,ma10/ma20_difff,ma10/ma15_difff,ma15/ma20_difff,close_difff,open_difff,high_difff,low_difff,volume_difff
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-10-09 17:00:00,1.47607,1.47538,1.47230,1.47292,3509,-0.000665,-0.000796,0.000132,1.473359,1.473326,52.364789,1.474557,0.999188,1.000022,0.999166,0.000235,0.000242,0.000007,0.00245,-0.00050,0.00241,0.00123,-316.0,0.000494,0.000532,-0.000038,0.00200,-0.00312,-0.00062,-0.00067,584.0
2009-10-09 18:00:00,1.47548,1.47119,1.47101,1.47534,3130,-0.000858,-0.000809,-0.000049,1.473276,1.472981,42.007252,1.474228,0.999354,1.000200,0.999154,0.000178,0.000166,-0.000011,-0.00419,0.00242,-0.00059,-0.00129,-379.0,0.000408,0.000413,-0.000004,-0.00174,0.00192,0.00182,-0.00006,-695.0
2009-10-09 19:00:00,1.47159,1.47036,1.46723,1.47121,3221,-0.001066,-0.000860,-0.000206,1.473140,1.472759,40.306500,1.473784,0.999563,1.000259,0.999304,0.000059,0.000209,0.000150,-0.00083,-0.00413,-0.00389,-0.00378,91.0,0.000375,0.000237,0.000139,-0.00502,-0.00171,-0.00448,-0.00507,-288.0
2009-10-09 20:00:00,1.47105,1.47000,1.46939,1.47035,1628,-0.001246,-0.000937,-0.000308,1.472908,1.472679,39.558392,1.473332,0.999713,1.000155,0.999557,-0.000104,0.000150,0.000253,-0.00036,-0.00086,-0.00054,0.00216,-1593.0,0.000358,-0.000045,0.000403,-0.00119,-0.00499,-0.00443,-0.00162,-1502.0
2009-10-09 21:00:00,1.47102,1.47070,1.46906,1.46998,1389,-0.001316,-0.001013,-0.000303,1.472749,1.472585,41.819629,1.472953,0.999862,1.000111,0.999750,-0.000044,0.000149,0.000193,0.00070,-0.00037,-0.00003,-0.00033,-239.0,0.000298,-0.000148,0.000446,0.00034,-0.00123,-0.00057,0.00183,-1832.0


In [ ]:
Xnew = X.copy()
cols = X.columns
i = 0
for c1 in cols:
  j = 0
  for c2 in cols:
    if i<j:
      Xnew[c1+'/'+c2] = Xnew[c1] / Xnew[c2]
      # Xnew[c1+'*'+c2] = Xnew[c1] * Xnew[c2]
    j += 1
  i+= 1

In [ ]:
import lightgbm
L = int(len(Xnew)*0.85)
Xtrain, Xtest = Xnew.iloc[:L], Xnew.iloc[L:]
Ytrain, Ytest = Y.iloc[:L], Y.iloc[L:]
lgb = lightgbm.LGBMClassifier(random_state=2020, n_estimators=10000, num_leaves=31, learning_rate=0.01, colsample_bytree=0.8,
                              importance_type='gain', verbose=1)
lgb.fit(Xtrain, Ytrain, eval_set=(Xtest, Ytest), verbose=100, early_stopping_rounds=50)
pred = lgb.predict(Xtest, num_iteration=lgb.best_iteration_)
print(accuracy_score(Ytest, pred))
print(confusion_matrix(Ytest, pred))

Training until validation scores don't improve for 50 rounds.
[100]	valid_0's multi_logloss: 0.74406
[200]	valid_0's multi_logloss: 0.697979
[300]	valid_0's multi_logloss: 0.679833
[400]	valid_0's multi_logloss: 0.670067
[500]	valid_0's multi_logloss: 0.663592
[600]	valid_0's multi_logloss: 0.659794
[700]	valid_0's multi_logloss: 0.657291
[800]	valid_0's multi_logloss: 0.655331
[900]	valid_0's multi_logloss: 0.654131
[1000]	valid_0's multi_logloss: 0.653279
[1100]	valid_0's multi_logloss: 0.652709
[1200]	valid_0's multi_logloss: 0.652305
[1300]	valid_0's multi_logloss: 0.651989
[1400]	valid_0's multi_logloss: 0.651854
[1500]	valid_0's multi_logloss: 0.651775
[1600]	valid_0's multi_logloss: 0.651585
[1700]	valid_0's multi_logloss: 0.651433
[1800]	valid_0's multi_logloss: 0.651291
Early stopping, best iteration is:
[1796]	valid_0's multi_logloss: 0.651252
0.7545278137128072
[[ 133   92  927]
 [ 125   95  960]
 [ 105   68 6771]]


In [ ]:
imp = pd.DataFrame({'Value':lgb.feature_importances_,'Feature':Xnew.columns})
imp = imp.sort_values('Value', ascending=False)
best_feats = list(imp['Feature'][:60].values)
imp.iloc[:20]

,Value,Feature
32,331491.442872,high/low
31,31732.347566,high/close
61,29176.076221,close/low
4,18513.773354,volume
409,16796.774051,ma15/ma20_diff/ma15/ma20_difff
17,16293.676528,ma15/ma20_diff
25,14683.408045,ma15/ma20_difff
33,12736.698418,high/open
22,12374.500305,volume_diff
162,12041.740225,volume/volume_diff


In [ ]:
Xnew_ = Xnew[best_feats]
Xtrain, Xtest = Xnew_.iloc[:L], Xnew_.iloc[L:]
lgb_ = lightgbm.LGBMClassifier(random_state=2020, n_estimators=10000, num_leaves=31, learning_rate=0.01, colsample_bytree=0.9,
                              importance_type='gain', verbose=1)
lgb_.fit(Xtrain, Ytrain, eval_set=(Xtest, Ytest), verbose=100, early_stopping_rounds=50)
pred = lgb_.predict(Xtest, num_iteration=lgb_.best_iteration_)
print(accuracy_score(Ytest, pred))
print(confusion_matrix(Ytest, pred))

Training until validation scores don't improve for 50 rounds.
[100]	valid_0's multi_logloss: 0.742678
[200]	valid_0's multi_logloss: 0.696126
[300]	valid_0's multi_logloss: 0.677822
[400]	valid_0's multi_logloss: 0.666867
[500]	valid_0's multi_logloss: 0.660965
[600]	valid_0's multi_logloss: 0.65732
[700]	valid_0's multi_logloss: 0.654817
[800]	valid_0's multi_logloss: 0.653315
[900]	valid_0's multi_logloss: 0.652571
[1000]	valid_0's multi_logloss: 0.652017
[1100]	valid_0's multi_logloss: 0.651635
[1200]	valid_0's multi_logloss: 0.651239
[1300]	valid_0's multi_logloss: 0.651143
[1400]	valid_0's multi_logloss: 0.650792
[1500]	valid_0's multi_logloss: 0.650649
[1600]	valid_0's multi_logloss: 0.650512
[1700]	valid_0's multi_logloss: 0.650407
Early stopping, best iteration is:
[1740]	valid_0's multi_logloss: 0.650376
0.7551746442432082
[[ 132   90  930]
 [ 122  108  950]
 [  94   85 6765]]


In [ ]:
res = pd.DataFrame({'actual':Ytest.values, 'pred':pred}, index=Ytest.index)
# res['actual'] = Ytest.values
res.tail(20)

,actual,pred
date,,
2020-04-24 04:00:00,so so,so so
2020-04-24 05:00:00,so so,so so
2020-04-24 06:00:00,so so,so so
2020-04-24 07:00:00,sell,so so
2020-04-24 08:00:00,sell,so so
2020-04-24 09:00:00,buy,so so
2020-04-24 10:00:00,buy,so so
2020-04-24 11:00:00,buy,so so
2020-04-24 12:00:00,buy,so so


In [ ]:
res.to_csv('/content/prediction.csv')

In [ ]:
XGB = xgboost.XGBClassifier(random_state=2020, n_estimators=200, max_depth=5, colsample_bytree=0.9)
XGB.fit(Xnew,Y)
w = XGB.feature_importances_
imp = pd.DataFrame({'feature':Xnew.columns, 'weight':w})
imp = imp.sort_values('weight', ascending=False)
print(imp[:8])

       feature    weight
3         open  0.073002
8         ma30  0.064298
1        close  0.057357
2          low  0.032627
7         ma10  0.028912
0         high  0.028168
35  open/sigal  0.019269
47  sigal/ma30  0.019229


In [ ]:
L = int(len(Xnew)*0.85)
Xtrain, Xtest = Xnew.iloc[:L,:], Xnew.iloc[L:,:]
Ytrain, Ytest = Y.iloc[:L], Y.iloc[L:]
Ytest.value_counts()

so so    985
sell     136
buy      125
Name: target, dtype: int64

In [ ]:
# from sklearn.utils import class_weight
# class_weights = list(class_weight.compute_class_weight('balanced',
#                                              np.unique(Ytrain),
#                                              Ytrain))

# w_array = np.ones(len(Ytrain), dtype = 'float')
# for i, val in enumerate(Ytrain):
#     w_array[i] = class_weights[val]

# xgb_classifier.fit(X, y, sample_weight=w_array)

In [ ]:
XGB = xgboost.XGBClassifier(random_state=2020, n_estimators=190, max_depth=5, colsample_bytree=0.9)
best_feats = list(imp['feature'].values[:6])
#pred = cross_val_predict(XGB, Xnew[best_feats], Y, cv=5)
XGB.fit(Xtrain[best_feats], Ytrain)#, sample_weight=w_array)
pred = XGB.predict(Xtest[best_feats])

# cnf_matrix = confusion_matrix(Ytest, pred,labels=['so so', 'sell', 'buy'])
# np.set_printoptions(precision=2)
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=['so so', 'sell', 'buy'], title='Confusion matrix')
print(accuracy_score(Ytest, pred))
print(confusion_matrix(Ytest, pred))

0.7784911717495987
[[  1   0 124]
 [  2   0 134]
 [ 16   0 969]]


In [ ]:
import pickle
file_name = "model.pkl"
pickle.dump(XGB, open(file_name, "wb"))